In [2]:
%run 0.0_Dependance.ipynb

In [3]:
df = pd.read_csv("train.csv",index_col=0)
df.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
id,,,,,,,,,,,,,,,,,,,
0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [4]:
df["N_Week"]=np.round((df['N_Days']/7),1)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)
df.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,...,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,N_Week,N_Year
id,,,,,,,,,,,,,,,,,,,,,
0,999,D-penicillamine,59.0,M,N,N,N,N,2.3,316.0,...,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D,142.7,2.74
1,2574,Placebo,52.7,F,N,N,N,N,0.9,364.0,...,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C,367.7,7.05
2,3428,Placebo,37.6,F,N,Y,Y,Y,3.3,299.0,...,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D,489.7,9.39
3,2576,Placebo,50.6,F,N,N,N,N,0.6,256.0,...,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C,368.0,7.06
4,788,Placebo,45.6,F,N,Y,N,N,1.1,346.0,...,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C,112.6,2.16


In [5]:
#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

In [5]:
df.head()

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,...,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status,N_Week,N_Year
id,,,,,,,,,,,,,,,,,,,,,
0,999,0,59.0,1,0,0,0,0,2.3,316.0,...,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,2,142.7,2.74
1,2574,1,52.7,0,0,0,0,0,0.9,364.0,...,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,0,367.7,7.05
2,3428,1,37.6,0,0,1,1,2,3.3,299.0,...,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,2,489.7,9.39
3,2576,1,50.6,0,0,0,0,0,0.6,256.0,...,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,0,368.0,7.06
4,788,1,45.6,0,0,1,0,0,1.1,346.0,...,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,0,112.6,2.16


# xgboost

In [ ]:
!pip install xgboost

In [6]:
param_grid = {
    # Nombre d'arbres dans le boosting
    'n_estimators': [50, 100, 250, 500, 1000],
    
    # Profondeur de chaque arbre
    'max_depth': [3, 6, 9, 12],
    
    # Taux d'apprentissage (eta)
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    
    # Minimum de perte de réduction nécessaire pour faire une partition supplémentaire sur un noeud de l'arbre
    'gamma': [0, 0.1, 0.5, 1],
    
    # Fraction des observations à être randomisées pour chaque arbre
    'subsample': [0.5, 0.75, 1],
    
    # Fraction de colonnes à être randomisées pour chaque arbre
    'colsample_bytree': [0.5, 0.75, 1],
    
    # Le minimum de poids nécessaire pour une enfant
    'min_child_weight': [1, 5, 10],
    
    # Regularisation L1 sur les poids
    'reg_alpha': [0, 0.1, 1],
    
    # Regularisation L2 sur les poids
    'reg_lambda': [0, 1, 10],
    
    # Choix de la fonction objectif, logloss pour la classification
    'objective': ['multi:softprob'],
    
    # Choix de la stratégie d'évaluation pour le boosting
    'eval_metric': ['mlogloss'],
    
    # Choix du booster
    'booster': ['gbtree'],
}


In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

joblib_path_suivi_metrique_1 = get_output_path_file("df_score_random.joblib")
joblib_path_suivi_metrique_2 = get_output_path_file("df_score_best_test.joblib")

target = df.Status
data = df.drop(columns=["Status","N_Days","N_Week"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#Modèle instancié
model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.1, objective="multi:softprob")

randomized_search_CV = RandomizedSearchCV(model, param_distributions=param_grid,n_iter=2000, cv=5, scoring='neg_log_loss',n_jobs=-1)
df_score_random,df_score_best_test = test_randomized_model(joblib_path_suivi_metrique_1, joblib_path_suivi_metrique_2, param_grid, model, randomized_search_CV)

Récupération des DataFrames existants
Début du randomized: RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                   param_distributions={'booster': ['gbtree'],
                

In [11]:
df_score_random

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std
0,250,9,0.01,0.1,0.75,1,10,1,1,multi:softprob,mlogloss,gbtree,-0.495877,0.007797
1,100,3,0.2,0.5,1,1,1,0.1,10,multi:softprob,mlogloss,gbtree,-0.448388,0.014140
2,1000,9,0.05,1,0.5,0.5,10,1,10,multi:softprob,mlogloss,gbtree,-0.440227,0.015089
3,500,12,0.2,0,0.75,0.75,5,0,0,multi:softprob,mlogloss,gbtree,-0.619045,0.030493
4,500,9,0.2,1,0.5,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.466344,0.015956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,500,3,0.05,0.1,0.5,0.5,1,0,0,multi:softprob,mlogloss,gbtree,-0.437674,0.015695
1996,1000,6,0.05,0.5,0.75,0.75,10,0.1,0,multi:softprob,mlogloss,gbtree,-0.460141,0.016867
1997,250,12,0.2,1,0.75,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.451227,0.016612
1998,500,12,0.01,0,1,0.75,10,0.1,1,multi:softprob,mlogloss,gbtree,-0.448577,0.013248


In [18]:
df_score_best_test

,Best_param,LogLoss,Accuracy,F1_weighted,F1_macro,Duree en sec randomized
0,"{'subsample': 0.5, 'reg_lambda': 0, 'reg_alpha...",0.44855,0.826692,0.815207,0.610022,3441.852242


In [15]:
from joblib import load,dump
dump(df_score_best_test,"df_score_best_test.joblib")

['df_score_best_test.joblib']

In [16]:
dump(df_score_random,"df_score_random.joblib")

['df_score_random.joblib']

In [17]:
df_score_random = load("df_score_random.joblib")
df_score_random

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std
0,250,9,0.01,0.1,0.75,1,10,1,1,multi:softprob,mlogloss,gbtree,-0.495877,0.007797
1,100,3,0.2,0.5,1,1,1,0.1,10,multi:softprob,mlogloss,gbtree,-0.448388,0.014140
2,1000,9,0.05,1,0.5,0.5,10,1,10,multi:softprob,mlogloss,gbtree,-0.440227,0.015089
3,500,12,0.2,0,0.75,0.75,5,0,0,multi:softprob,mlogloss,gbtree,-0.619045,0.030493
4,500,9,0.2,1,0.5,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.466344,0.015956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,500,3,0.05,0.1,0.5,0.5,1,0,0,multi:softprob,mlogloss,gbtree,-0.437674,0.015695
1996,1000,6,0.05,0.5,0.75,0.75,10,0.1,0,multi:softprob,mlogloss,gbtree,-0.460141,0.016867
1997,250,12,0.2,1,0.75,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.451227,0.016612
1998,500,12,0.01,0,1,0.75,10,0.1,1,multi:softprob,mlogloss,gbtree,-0.448577,0.013248
